In [7]:
from database.adatabase import ADatabase
from extractor.alp_client_extractor import ALPClientExtractor
from extractor.tiingo_extractor import TiingoExtractor
from processor.processor import Processor as processor
from datetime import datetime, timedelta
from tqdm import tqdm 
from dotenv import load_dotenv
load_dotenv()
import os
import pandas as pd
from time import sleep
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import argrelextrema


In [8]:
# Initialize parameters
window = 300
gain_window = 60
extractor = ALPClientExtractor(os.getenv("APCAKEY"), os.getenv("APCASECRET"))
tickers = ["AAVE", "AVAX", "BAT", "BTC", "CRV", "DOGE", "DOT", "ETH", "GRT", "LINK", "LTC", "MKR", "SHIB", "SUSHI", "UNI", "XTZ", "YFI"]
tickers = [x + "/USD" for x in tickers]

prices = []
start = datetime.now() - timedelta(days=1)
end = datetime.now()

# Loop through each ticker to fetch prices and calculate metrics
for ticker in tickers:
    response = extractor.crypto_interval(ticker, start)
    price = pd.DataFrame(response["bars"][ticker]).rename(columns={"c": "adjclose", "t": "date"})[["date", "adjclose"]]
    
    # Convert date and sort
    price["date"] = pd.to_datetime(price["date"])
    price = price.sort_values("date")
    
    # Calculate metrics
    price["rolling_return"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"]) / price["adjclose"])
    price["bollinger"] = ((price["adjclose"].rolling(window).mean() - price["adjclose"].rolling(window).std() - price["adjclose"]) / price["adjclose"])
    price["return"] = price["adjclose"].pct_change(gain_window)
    price["avg_return"] = price["return"].pct_change(gain_window).rolling(window).mean()
    price["coev"] = (price["adjclose"].rolling(window).mean() / price["adjclose"].rolling(window).std())
    price["ticker"] = ticker
    prices.append(price)

# Combine all prices into a single DataFrame
sim = pd.concat(prices).sort_values("date").dropna()

In [9]:
metrics = [
    "rolling_return",
    "bollinger",
    "return",
    "avg_return",
    "coev"
]

In [32]:
import copy
analysis = []
date_range = list(sim["date"].unique())
base_position = {"date":date_range[0],"ticker":"","quantity":0,"adjclose":0,"buy_price":0,"buy_date":date_range[0]}
for metric in metrics:
    for ascending in [True,False]:
        cash = 100
        position = copy.deepcopy(base_position)
        cashes = []
        for date in tqdm(date_range):
            try:
                position = copy.deepcopy(position)
                position["date"] = date
                today = sim[sim["date"]==date]
                if today.index.size >= 0:
                    opportunity = today.sort_values(metric,ascending=ascending).iloc[0]
                    if position["ticker"] != "":
                        position["adjclose"] = today[today["ticker"]==position["ticker"]]["adjclose"].iloc[0].item()
                        pnl_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] >= 0.005)
                        loss_req = ((position["adjclose"] - position["buy_price"]) / position["buy_price"] <= -0.005)
                        hpr_req = (date - position["buy_date"]).days > float(60/(24*60))
                        if pnl_req:
                            try:
                                cash = position["adjclose"] * position["quantity"] * 0.99
                                cashes.append(cash)
                                position = copy.deepcopy(base_position)
                                position["date"] = date
                                position["ticker"] = opportunity["ticker"]
                                position["quantity"] = cash / opportunity["adjclose"] 
                                position["adjclose"] = opportunity["adjclose"]
                                position["buy_price"] = opportunity["adjclose"]
                                position["buy_date"] = date
                            except Exception as e:
                                continue
                    else:
                        position["date"] = date
                        position["ticker"] = opportunity["ticker"]
                        position["quantity"] = cash / opportunity["adjclose"] 
                        position["adjclose"] = opportunity["adjclose"]
                        position["buy_price"] = opportunity["adjclose"]
                        position["buy_date"] = date
            except Exception as e:
                continue
        position["metric"] = metric
        position["ascending"] = ascending
        position["pv"] = position["adjclose"] * position["quantity"]
        analysis.append(copy.deepcopy(position))

100%|████████████████████████████████████████████████████████████████████████████████████████| 1835/1835 [00:07<00:00, 253.22it/s]


In [33]:
a = pd.DataFrame(analysis)
a.sort_values("pv",ascending=False)

,date,ticker,quantity,adjclose,buy_price,buy_date,metric,ascending,pv
2,2024-09-27 14:29:00+00:00,DOGE/USD,817.376129,0.126745,0.122590,2024-09-27 07:36:00+00:00,bollinger,True,103.598337
4,2024-09-27 14:29:00+00:00,DOGE/USD,817.376129,0.126745,0.122590,2024-09-27 07:36:00+00:00,return,True,103.598337
8,2024-09-27 14:29:00+00:00,DOGE/USD,817.376129,0.126745,0.122590,2024-09-27 07:36:00+00:00,coev,True,103.598337
6,2024-09-27 14:29:00+00:00,CRV/USD,324.888592,0.317580,0.308420,2024-09-27 07:36:00+00:00,avg_return,True,103.177957
1,2024-09-27 14:29:00+00:00,LTC/USD,1.454590,70.408500,68.886850,2024-09-27 07:36:00+00:00,rolling_return,False,102.415517
3,2024-09-27 14:29:00+00:00,LTC/USD,1.454590,70.408500,68.886850,2024-09-27 07:36:00+00:00,bollinger,False,102.415517
9,2024-09-27 14:29:00+00:00,LTC/USD,1.454590,70.408500,68.886850,2024-09-27 07:36:00+00:00,coev,False,102.415517
7,2024-09-27 14:29:00+00:00,GRT/USD,531.684931,0.189315,0.188462,2024-09-27 07:36:00+00:00,avg_return,False,100.655933
0,2024-09-27 14:29:00+00:00,MKR/USD,0.060010,1668.013500,1669.751500,2024-09-27 07:36:00+00:00,rolling_return,True,100.097842
5,2024-09-27 14:29:00+00:00,MKR/USD,0.060010,1668.013500,1669.751500,2024-09-27 07:36:00+00:00,return,False,100.097842
